<a href="https://colab.research.google.com/github/abogaziah/Pong_DQN/blob/master/Policy_gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch import nn
import torch.nn.functional as F
import gym
from torch.distributions.categorical import Categorical
import statistics
from matplotlib import pyplot as plt

In [0]:
class policy(nn.Module):
  def __init__(self, D_in, H, D_out):
    super(policy, self).__init__()
    self.fc1 = torch.nn.Linear(D_in,H)
    self.fc2 = torch.nn.Linear(H,D_out)

  def forward(self, state):
    x = F.relu(self.fc1(state))
    x = F.softmax(self.fc2(x),dim=0)
    return x

In [0]:
env = gym.make('CartPole-v0')
din, h, dout = env.observation_space.shape[0], 128 , env.action_space.n
policy = policy(din, h, dout)
episodes= 500
reward_buffer=[]
for episode in range(episodes):
  state = env.reset()
  done = False 
  rewards=[]
  prob_buffer=[]
  while not done:
    probs = policy.forward(torch.from_numpy(state).float())
    m = Categorical(probs)
    action = m.sample()
    state, reward, done, _ = env.step(action.item())
    rewards.append(reward)
    loss = -m.log_prob(action) * reward
    loss.backward()
  reward_buffer.append(statistics.mean(rewards))
plt.plot(reward_buffer)
plt.show()